In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt

from HexDetector import hex_radius_inner_to_outer
from analysis_functions import *

set_matplotlib_style()

In [ ]:
run_number = 358
pool_number = 8
# run_path = f'/home/akallits/Documents/PicoAnalysis/Saclay_Analysis/data/2023_April_h4/processedTrees/ParameterTrees/Run{run_number}-Pool{pool_number}_treeParam.root'
# run_path = f'/data/akallits/Saclay_Analysis/data/data/2023_April_h4/processedTrees/ParameterTrees/Run{run_number}-Pool{pool_number}_treeParam.root'
# run_path = f'/local/home/dn277127/Bureau/picosec/Run{run_number}-Pool{pool_number}_treeParam.root'
run_path = f'/home/dylan/Desktop/picosec/data/2024_June_h4/processedTrees/ParameterTrees/Run{run_number}-Pool{pool_number}_treeParam.root'

oscilloscope_file = f'oscilloscope_time_offsets_2024_June_h4.root'

In [ ]:
tree = get_tree(run_path, 'ParameterTree')

In [ ]:
# event_start, event_end = 4416, None
min_ampl, max_ampl = -1, 1  # V
min_charge, max_charge = -0, 1.0 # fC
min_charge_tot, max_charge_tot = -0, 100.0  # fC
min_echarge, max_echarge = -0, 10.0  # fC
xy_range_min, xy_range_max = -100, 100  # mm
mm_saturation_voltage_buffer = 0.01 # V Place cut this percent lower than max max amplitude point
chi2_quality = 3
# chi2_quality = 1
# sat_range = None
sat_range = [-1e9, 1e9]
# sat_range = [-0.04, 1.0]

time_walk_nbins = 50  # For SAT vs charge
n_charge_bins = 50  # For resolution vs charge fits

mcp_min_avg_charge_per_2d_bin = 0.01
mm_min_avg_charge_per_2d_bin = 11
mcp_min_tracks_per_2d_bin = 20
mm_min_tracks_per_2d_bin = 10
mcp_2d_bin_width = 0.5
mm_2d_bin_width = 0.5

time_walk_func = time_walk_double_exponential
p0_time_walk = [1.0, -0.1, 0.5, -0.1, 4] #ampl1, decay1, amp2, decay2, bsl

p0_resol = [30, -0.1, 10, -0.05, 20]
resol_func = time_walk_double_exponential
resolution_correction_to_use = 'Corrected Gaus'  # Correction to use moving forward

In [ ]:
mcp_channel = 'C1'
mm_channel = 'C4'
channels = [mcp_channel, mm_channel]
global_vars = ['eventNo', 'chi2track', 'SRSNo']
channel_vars = ['hitX', 'hitY', 'npeaks']
peak_param_channel_vars = ['ampl', 'dampl', 'charge', 'echarge', 'echargefit', 'echargefixed', 'totcharge', 'totchargefixed', 'risetime', 'tfit20','tfit20_nb', 'tnaive20', 'sigmoidR[4]']

branches = []
for channel in channels:
    for channel_var in channel_vars:
        branches.append(f'{channel_var}_{channel}')
    for var in peak_param_channel_vars:
        branches.append(f'peakparam_{channel}/peakparam_{channel}.{var}')
for var in global_vars:
    branches.append(var)
df = get_df_branches(tree, branches)


# if event_start is not None:
#     df = df[df['eventNo'] >= event_start]
# if event_end is not None:
#     df = df[df['eventNo'] <= event_end]

## Load the time offsets for this run

In [ ]:
df = get_rel_time_offsets(df, oscilloscope_file, run_number, pool_number, mcp_channel, mm_channel)

# Select The Best Tracks in Each Event


Before the following cell, each event can have multiple tracks (chi2track, hitX/Y_Cn). After the below cell, each event will only have one track. All track columns will be pd Series of floats, instead of awkward arrays


In [ ]:
print(f'Start processing: {df["eventNo"].shape[0]} events')
get_best_tracks(df, channels, chi2_quality, plot=True)

In [ ]:
fig_x, ax_x = plt.subplots(figsize=(8, 4))
ax_x.scatter(df['SRSNo'], df['hitX_C1'], s=0.5, alpha=0.05)
ax_x.set_xlabel('SRSNo')
ax_x.set_ylabel('X [mm]')

fig_y, ax_y = plt.subplots(figsize=(8, 4))
ax_y.scatter(df['SRSNo'], df['hitY_C1'], s=0.5, alpha=0.05)
ax_y.set_xlabel('SRSNo')
ax_y.set_ylabel('Y [mm]')

fig_xy, ax_xy = plt.subplots(figsize=(8, 4))
ax_xy.scatter(df['hitX_C1'], df['hitY_C1'], s=0.5, alpha=0.05)
ax_xy.set_xlabel('X [mm]')
ax_xy.set_ylabel('Y [mm]')

import plotly.express as px
import pandas as pd
# Assume df has columns 'hitX_C1', 'hitY_C1', and 'SRSNo'
n_events = len(df)
step_size = 800
n_steps = int(np.ceil(n_events / step_size))

# Add a time step column to df based on event number
df_plotly = df.copy()
df_plotly['time'] = df_plotly['SRSNo'] // step_size

# Plot with plotly
import plotly.express as px

fig = px.scatter(
    df_plotly,
    x="hitX_C1",
    y="hitY_C1",
    animation_frame="time",
    title="2D Hits Over Time (Grouped by 1000 Events)",
    range_x=[df['hitX_C1'].min(), df['hitX_C1'].max()],
    range_y=[df['hitY_C1'].min(), df['hitY_C1'].max()],
    height=600,
    width=600
)

fig.update_traces(marker=dict(size=5, opacity=0.7))
# Completely disable all animations between frames
# Modify existing play button to remove transitions and enable redraw
fig.layout.updatemenus[0].buttons[0].args[1]['frame'] = {'duration': 500, 'redraw': True}
fig.layout.updatemenus[0].buttons[0].args[1]['transition'] = {'duration': 0}

# Optional: update pause button too if desired (second button)
if len(fig.layout.updatemenus[0].buttons) > 1:
    fig.layout.updatemenus[0].buttons[1].args[1]['frame'] = {'duration': 0, 'redraw': True}
    fig.layout.updatemenus[0].buttons[1].args[1]['transition'] = {'duration': 0}
fig.show()

In [ ]:
for channel in channels:
    get_single_peak(df, channel)

In [ ]:
event_start_number = get_run_event_start(df)
print(event_start_number)
# event_start, event_end = 4416, None
event_start, event_end = event_start_number, None

if event_start is not None:
    df = df[df['eventNo'] >= event_start]
if event_end is not None:
    df = df[df['eventNo'] <= event_end]

In [ ]:
# Loop over channels
mcp_bins = np.arange(0, 1.0, 0.01)
ebins = np.arange(0, 10, 0.2)
totbins = np.arange(0, 100, 2)

for channel in [mcp_channel, mm_channel]:
    if channel == mcp_channel:
        charge_types = ["charge"]
    else:
        charge_types = ["echargefit", "echarge", "echargefixed", "totchargefixed", "totcharge"]

    num_charges = len(charge_types)

    fig, axes = plt.subplots(1, num_charges, figsize=(12,6), sharey=True)

    # Ensure axes is always iterable (wrap in list if only one subplot)
    if num_charges == 1:
        axes = [axes]

    # fig.suptitle(f"Histograms of Charges for {channel}", fontsize=14)

    for i, charge_type in enumerate(charge_types):
        key = f'peakparam_{channel}/peakparam_{channel}.{charge_type}'
        charge_data = df[key]

        if "totcharge" in charge_type:
            filtered_data = charge_data[(charge_data > min_charge_tot) & (charge_data < max_charge_tot)]
            binning = totbins
        elif "echarge" in charge_type:
            filtered_data = charge_data[(charge_data > min_echarge) & (charge_data < max_echarge)]
            binning = ebins
        else:
            filtered_data = charge_data[(charge_data > min_charge) & (charge_data < max_charge)]
            binning = ebins
            if channel == mcp_channel:
                binning = mcp_bins

        n_overflows = np.sum(charge_data > binning[-1])
        axes[i].hist(filtered_data, bins=binning, alpha=0.7, color='blue', edgecolor='black', zorder=2)
        axes[i].set_title(charge_type)
        axes[i].set_xlabel('Charge [pC]')
        axes[i].grid(True, zorder=0)
        # Add overflow text
        axes[i].text(
            0.95, 0.9,  # Position (relative to axes, 95% right, 90% up)
            f'Overflows: {n_overflows}',
            transform=axes[i].transAxes,  # Use axes coordinates (0,0 is bottom-left, 1,1 is top-right)
            fontsize=12, color='red', ha='right', va='top',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')  # Add a background box
        )
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout
plt.savefig(f'charge_histograms_{channel}.pdf', dpi=300)
plt.savefig(f'charge_histograms_{channel}.png', dpi=300)

In [ ]:
channel_charge_types = {
    mcp_channel: ["charge"],
    mm_channel: ["echargefit", "echarge", "echargefixed", "totchargefixed", "totcharge"],
}

# Loop through the channels you want to plot
for channel in channels:
    charge_types = channel_charge_types[channel]  # Select charge types based on the channel

    fig, axs = plt.subplots(ncols=2, figsize=(8, 4))  # Create figure with 2 subplots (ncols=2)

    # Loop over each charge type for the current channel
    for charge_type in charge_types:
        key = f'peakparam_{channel}/peakparam_{channel}.{charge_type}'
        charge_data = df[key]

        # Apply filters based on the charge type (e.g., total charge vs. echarge)
        if "totcharge" in charge_type:
            filtered_data = charge_data[(charge_data > min_charge_tot) & (charge_data < max_charge_tot)]
            binning = totbins
            ax_num = 0  # First subplot (totcharge)
        elif "echarge" in charge_type:
            filtered_data = charge_data[(charge_data > min_echarge) & (charge_data < max_echarge)]
            binning = ebins
            ax_num = 1
        else:
            filtered_data = charge_data[(charge_data > min_charge) & (charge_data < max_charge)]
            binning = ebins
            ax_num = 1  # Second subplot (echarge)
            if channel == mcp_channel:
                binning = mcp_bins  # Special binning for mcp

        # Plot the histogram
        n, bins, _ = axs[ax_num].hist(filtered_data, bins=binning, alpha=0.7, label=charge_type,
                                      zorder=2, histtype='step', linewidth=3)

        # Optional: Adding labels on top of histogram lines (bins)
        for i in range(len(bins)-1):
            # Find the position to put the label (at the middle of each bin)
            bin_center = (bins[i] + bins[i+1]) / 2
            count = n[i]  # Get the count (height) of the histogram at this bin
            # axs[ax_num].text(bin_center, count, f'{int(count)}', ha='center', va='bottom', fontsize=8)

    # Customize the plots
    axs[0].set_xlabel('Charge [pC]')
    axs[0].grid(True)
    axs[1].set_xlabel('Charge [pC]')
    axs[1].grid(True)

    # Add legends to the subplots
    axs[0].legend(title="Charge Types")
    axs[1].legend(title="Charge Types")

    # Set title for the current channel
    # fig.suptitle(f"Histograms of Charges for {channel}", fontsize=14)

    # Adjust the layout to avoid overlap
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Show plot for the current channel
    plt.show()

In [ ]:
unique_offsets, unique_offset_counts = np.unique(df['rel_time_offset'], return_counts=True)
print(f'Unique offets: {unique_offsets}')
print(f'Unique offset counts: {unique_offset_counts}')
fig, ax = plt.subplots()
ax.hist(df['rel_time_offset']);

In [ ]:
time_col = 'tfit20_nb'
df[f'{mm_channel}_time_diff'] = df[f'peakparam_{mm_channel}/peakparam_{mm_channel}.{time_col}'] - df[f'peakparam_{mcp_channel}/peakparam_{mcp_channel}.{time_col}'] - df['rel_time_offset']

filters = {channel: {} for channel in channels}
for channel in channels:
    
    tot_charge_data = df[f'peakparam_{channel}/peakparam_{channel}.totcharge']
    filters[channel].update({'totcharge_cut': (tot_charge_data > min_charge_tot) & (tot_charge_data < max_charge_tot)})

    echarge_data = df[f'peakparam_{channel}/peakparam_{channel}.echarge']
    filters[channel].update({'echarge_cut': (echarge_data > min_echarge) & (echarge_data < max_echarge)})

    echarge_fit_data = df[f'peakparam_{channel}/peakparam_{channel}.echargefit']
    filters[channel].update({'echargefit_cut': (echarge_fit_data > min_echarge) & (echarge_fit_data < max_echarge)})

    charge_data = df[f'peakparam_{channel}/peakparam_{channel}.charge']
    filters[channel].update({'charge_cut': (charge_data > min_charge_tot) & (charge_data < max_charge_tot)})

    ampl_data = df[f'peakparam_{channel}/peakparam_{channel}.ampl']
    filters[channel].update({'ampl_cut': (ampl_data > min_ampl) & (ampl_data < max_ampl)})

    dampl_data = df[f'peakparam_{channel}/peakparam_{channel}.dampl']
    filters[channel].update({'dampl_cut': (dampl_data > min_ampl) & (dampl_data < max_ampl)})
    # filters[channel].update({'saturation_cut': dampl_data < mm_saturation_voltage})
    mm_saturation_voltage = np.max(dampl_data) * (1 - mm_saturation_voltage_buffer)
    filters[channel].update({'saturation_cut': dampl_data < mm_saturation_voltage})


    #
    # echarge_fit_data = df[f'peakparam_{channel}/peakparam_{channel}.echargefit']
    # filters[channel].update({'echargefit_cut': (echarge_fit_data > min_charge) & (echarge_fit_data < max_charge)})

    x_data = df[f'hitX_{channel}']
    y_data = df[f'hitY_{channel}']
    
    xy_filter_cut = (x_data > xy_range_min) & (x_data < xy_range_max) & (y_data > xy_range_min) & (y_data < xy_range_max)
    filters[channel].update({'xy_cut': xy_filter_cut})
    
time_diff_data = df[f'{mm_channel}_time_diff']
filters[mm_channel].update({'time_diff_na_cut': ~pd.isna(time_diff_data)})
time_diff_median = np.nanmedian(time_diff_data)
filters[mm_channel].update({'time_diff_range_cut': (time_diff_data > time_diff_median + sat_range[0]) & (time_diff_data < time_diff_median + sat_range[1])})
print(f'Time diff median={time_diff_median}, cuts=[{time_diff_median + sat_range[0]}, {time_diff_median + sat_range[1]}]')

# Get filtered dataframes for each channel -- charge filter
mcp_charge_filtered_df = df[filters[mcp_channel]['charge_cut']]
mm_charge_filtered_df = df[filters[mm_channel]['totcharge_cut']]

x_mcp_filtered = mcp_charge_filtered_df[f'hitX_{mcp_channel}']
y_mcp_filtered = mcp_charge_filtered_df[f'hitY_{mcp_channel}']

In [ ]:
fig, axes = plt.subplots(nrows=len(channels), ncols=1, figsize=(8, 10))  # 3 rows, 1 column

for i, channel in enumerate(channels):
    # Get first charge or total charge based on the channel
    if channel == mcp_channel:
        first_charges_unfiltered = df[f'peakparam_{channel}/peakparam_{channel}.charge']
        df_filter = df[filters[channel]['charge_cut']]
        first_charges_filtered = df_filter[f'peakparam_{channel}/peakparam_{channel}.charge']
    else:
        first_charges_unfiltered = df[f'peakparam_{channel}/peakparam_{channel}.totcharge']
        df_filter = df[filters[channel]['totcharge_cut']]
        first_charges_filtered = df_filter[f'peakparam_{channel}/peakparam_{channel}.totcharge']

    # Convert to numpy arrays for filtering
    n_overflows = np.sum(first_charges_unfiltered < min_charge_tot) + np.sum(first_charges_unfiltered > max_charge_tot)  # Overflow count
    n_in_range = len(first_charges_filtered)
    n_total = n_in_range + n_overflows

    # Print outliers information
    print(f'{channel} Number of outliers: {n_overflows} of {n_total}')
    print(f'{channel} Number after filter: {len(first_charges_filtered)} of {n_total}')
    print(f'{channel} channel: min = {np.nanmin(first_charges_filtered)}, max = {np.nanmax(first_charges_filtered)}')

    # Plot each channel's charge histogram
    ax = axes[i]  # Use the appropriate axis in the grid
    ax.hist(first_charges_filtered, bins=100, alpha=0.7, color='blue', edgecolor='black', zorder=2)

    # Set title based on the channel
    if channel == mcp_channel:
        ax.set_title(f'Charge of first peak in {channel} channel')
    else:
        ax.set_title(f'Tot charge of first peak in {channel} channel')

    # Set labels
    ax.set_xlabel('Total Charge [pC]')
    ax.set_ylabel('Counts')

    # Add overflow text
    ax.text(
        0.95, 0.9,  # Position (relative to axes, 95% right, 90% up)
        f'Overflows: {n_overflows}',
        transform=ax.transAxes,  # Use axes coordinates (0,0 is bottom-left, 1,1 is top-right)
        fontsize=12, color='red', ha='right', va='top',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')  # Background box
    )

    ax.grid(True, zorder=0)

# Adjust layout to avoid overlap and ensure titles fit
fig.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to avoid overlap

In [ ]:
fig, axes = plt.subplots(nrows=len(channels), ncols=2, figsize=(12, 10))  # 3 rows, 2 columns

for i, channel in enumerate(channels):
    # First peak amplitude histogram
    first_ampls_unfiltered = df[f'peakparam_{channel}/peakparam_{channel}.ampl']
    df_filter = df[filters[channel]['ampl_cut']]
    first_ampls_filtered = df_filter[f'peakparam_{channel}/peakparam_{channel}.ampl']

    n_overflows = np.sum(first_ampls_unfiltered < min_ampl) + np.sum(first_ampls_unfiltered > max_ampl)  # Overflow count
    n_in_range = len(first_ampls_filtered)
    n_total = n_in_range + n_overflows

    # Plot first_ampls_filtered histogram
    ax = axes[i, 0]  # Left column
    ax.hist(first_ampls_filtered, bins=100, alpha=0.7, color='blue', edgecolor='black', zorder=2)
    ax.set_title(f'{channel} - Amplitude')
    ax.set_xlabel('Amplitude [V]')
    ax.set_ylabel('Counts')
    ax.grid(True, zorder=0)

    # Add overflow text for first_ampls
    ax.text(
        0.95, 0.9,  # Position (relative to axes, 95% right, 90% up)
        f'Overflows: {n_overflows}',
        transform=ax.transAxes,  # Use axes coordinates (0,0 is bottom-left, 1,1 is top-right)
        fontsize=12, color='red', ha='right', va='top',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')  # Background box
    )

    # For C2, add the second amplitude (dampl) plot
    if channel == mm_channel:
        first_dampls_unfiltered = df[f'peakparam_{channel}/peakparam_{channel}.dampl']
        df_filter = df[filters[channel]['dampl_cut']]
        first_dampls_filtered = df_filter[f'peakparam_{channel}/peakparam_{channel}.dampl']

        n_overflows = np.sum(first_dampls_unfiltered < min_ampl) + np.sum(first_dampls_unfiltered > max_ampl)  # Overflow count
        n_in_range = len(first_dampls_filtered)
        n_total = n_in_range + n_overflows

        # Plot first_dampls_filtered histogram
        ax = axes[i, 1]  # Right column
        ax.hist(first_dampls_filtered, bins=100, alpha=0.7, color='blue', edgecolor='black', zorder=2)
        ax.set_title(f'{channel} - Amplitude Fit')
        ax.set_xlabel('Amplitude [V]')
        ax.set_ylabel('Counts')
        ax.grid(True, zorder=0)

        # Add overflow text for first_dampls
        ax.text(
            0.95, 0.9,  # Position (relative to axes, 95% right, 90% up)
            f'Overflows: {n_overflows}',
            transform=ax.transAxes,  # Use axes coordinates (0,0 is bottom-left, 1,1 is top-right)
            fontsize=12, color='red', ha='right', va='top',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')  # Background box
        )

# Adjust layout
fig.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to avoid overlap
plt.savefig('amplitude_histograms.pdf', dpi=300)

In [ ]:
channels = filters.keys()  # Get all available channels
num_channels = len(channels)

fig, axes = plt.subplots(nrows=1, ncols=num_channels, figsize=(5*num_channels, 5))

if num_channels == 1:
    axes = [axes]  # Ensure `axes` is iterable when there's only one subplot

for ax, ch in zip(axes, channels):
    df_filter = df[filters[ch]['ampl_cut']]
    ampl_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.ampl']
    dampl_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.dampl']

    ax.scatter(ampl_filtered, dampl_filtered, s=1)
    ax.set_title(f'Channel {ch}')
    ax.set_xlabel('Amplitude-Fit [V]')
    ax.set_ylabel('Amplitude-Data [V]')
# fig.text(0.5, 0.02, 'Amplitude-Fit [V]', ha='center', fontsize=12)
plt.tight_layout()

In [ ]:
compare_ch_ampl = [ch for ch in filters.keys() if ch != mcp_channel]

num_plots = len(compare_ch_ampl)
fig, axes = plt.subplots(nrows=1, ncols=num_plots, figsize=(6*num_plots, 5), sharex=True, sharey=True)

if num_plots == 1:
    axes = [axes]  # Ensure `axes` is iterable when there's only one subplot

# Filter MCP data once (to avoid redundant computation)
df_filter_mcp = df[filters[mcp_channel]['ampl_cut']]
mcp_ampl_filtered = df_filter_mcp[f'peakparam_{mcp_channel}/peakparam_{mcp_channel}.ampl']

for ax, ch in zip(axes, compare_ch_ampl):
    # Apply filter for the other channel
    df_filter_ch = df[filters[ch]['ampl_cut']]
    ch_ampl_filtered = df_filter_ch[f'peakparam_{ch}/peakparam_{ch}.ampl']

    # Plot histograms
    ax.hist(mcp_ampl_filtered, bins=100, histtype='step', color='black', lw=2, label='C1-MCP')
    ax.hist(ch_ampl_filtered, bins=100, histtype='step', color='red', lw=2, label=f'{ch} - MM-Pad')

    # Set title and legend
    ax.set_title(f'C1 vs {ch}')
    ax.set_xlabel('Amplitude-Fit [V]')
    ax.legend()

plt.tight_layout()

In [ ]:
# Get all channels except C1
compare_ch_charge = [ch for ch in filters.keys() if ch != mcp_channel]

num_plots = len(compare_ch_charge)
print(num_plots)
fig, axes = plt.subplots(nrows=2, ncols=num_plots, figsize=(6*num_plots, 10), sharex=True, sharey=False)
print(axes.shape)

if num_plots == 1:
    axes = [axes]# Ensure `axes` is iterable when there's only one subplot
    print(axes)
# Filter C1 data once (to avoid redundant computation)
df_filter_mcp = df[filters[mcp_channel]['charge_cut']]
mcp_ch_filtered = df_filter_mcp[f'peakparam_{mcp_channel}/peakparam_{mcp_channel}.charge']

for col, ch in enumerate(compare_ch_charge):
    # Apply filter for the other channel
    df_filter_ch = df[filters[ch]['echargefit_cut']]
    ch_ech_filtered = df_filter_ch[f'peakparam_{ch}/peakparam_{ch}.echargefit']

    # Linear scale histogram (first row)
    axes[col][0].hist(mcp_ch_filtered, bins=100, histtype='step', color='black', lw=2, label='C1-MCP')
    axes[col][0].hist(ch_ech_filtered, bins=100, histtype='step', color='red', lw=2, label=f'{ch}-MM-Pad')
    axes[col][0].set_title(f'C1 vs {ch}')
    axes[col][0].set_xlabel('e-peak Charge [pC]')
    axes[col][0].legend()

    # Log scale histogram (second row)
    axes[col][1].hist(mcp_ch_filtered, bins=100, histtype='step', color='black', lw=2, label='C1-MCP')
    axes[col][1].hist(ch_ech_filtered, bins=100, histtype='step', color='red', lw=2, label=f'{ch}-MM-Pad')
    axes[col][1].set_yscale('log')  # Set log scale for y-axis
    axes[col][1].set_xlabel('e-peak Charge [pC]')
    axes[col][1].legend()

plt.tight_layout()

In [ ]:
# Get all channels except C1
compare_ch_totcharge = [ch for ch in filters.keys() if ch != mcp_channel]

num_plots = len(compare_ch_totcharge)
fig, axes = plt.subplots(nrows=2, ncols=num_plots, figsize=(6*num_plots, 10), sharex=True, sharey=False)

if num_plots == 1:
    axes = [axes]  # Ensure `axes` is iterable when there's only one subplot

# Filter C1 data once (to avoid redundant computation)
df_filter_mcp = df[filters[mcp_channel]['charge_cut']]
mcp_ch_filtered = df_filter_mcp[f'peakparam_{mcp_channel}/peakparam_{mcp_channel}.charge']

for col, ch in enumerate(compare_ch_totcharge):
    # Apply filter for the other channel
    df_filter_ch = df[filters[ch]['totcharge_cut']]
    ch_ech_filtered = df_filter_ch[f'peakparam_{ch}/peakparam_{ch}.totcharge']

    # Linear scale histogram (first row)
    axes[col][0].hist(mcp_ch_filtered, bins=100, histtype='step', color='black', lw=2, label='C1-MCP')
    axes[col][0].hist(ch_ech_filtered, bins=100, histtype='step', color='red', lw=2, label=f'{ch}-MM-Pad')
    axes[col][0].set_title(f'C1 vs {ch}')
    axes[col][0].set_xlabel('Total Charge [pC]')
    axes[col][0].legend()

    # Log scale histogram (second row)
    axes[col][1].hist(mcp_ch_filtered, bins=100, histtype='step', color='black', lw=2, label='C1-MCP')
    axes[col][1].hist(ch_ech_filtered, bins=100, histtype='step', color='red', lw=2, label=f'{ch}-MM-Pad')
    axes[col][1].set_yscale('log')  # Set log scale for y-axis
    axes[col][1].set_xlabel('Total Charge [pC]')
    axes[col][1].legend()

plt.tight_layout()

In [ ]:
channels = filters.keys()  # Get all available channels
num_channels = len(channels)

fig, axes = plt.subplots(nrows=1, ncols=num_channels, figsize=(5*num_channels, 5))

if num_channels == 1:
    axes = [axes]  # Ensure `axes` is iterable when there's only one subplot

for ax, ch in zip(axes, channels):
    df_filter = df[filters[ch]['ampl_cut']]
    ampl_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.ampl']
    dampl_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.dampl']


    df_filter_saturation = df[filters[ch]['ampl_cut'] & ~filters[ch]['saturation_cut']]
    ampl_saturation = df_filter_saturation[f'peakparam_{ch}/peakparam_{ch}.ampl']
    dampl_saturation = df_filter_saturation[f'peakparam_{ch}/peakparam_{ch}.dampl']



    ax.scatter(ampl_filtered, dampl_filtered, s=1)

    if ch != mcp_channel:
        ax.scatter(ampl_saturation, dampl_saturation, s=2, color='red', label=f'Saturation {mm_channel}')
        ax.axhline(mm_saturation_voltage, color='green', linestyle='-', label='Saturation Voltage')

    ax.set_title(f'Channel {ch}')
    ax.set_xlabel('Amplitude-Fit [V]')
    ax.set_ylabel('Amplitude-Data [V]')
# fig.text(0.5, 0.02, 'Amplitude-Fit [V]', ha='center', fontsize=12)
plt.tight_layout()

In [ ]:
channels = [ch for ch in filters.keys() if ch != mcp_channel]
num_channels = len(channels)
print(num_channels)

fig, axs = plt.subplots(nrows=num_channels, ncols=3, figsize=(18, 6*num_channels), sharex=False, sharey=False)

if num_channels == 1:
    axs = [axs]  # Ensure axs is iterable when there's only one channel

# Loop over each channel
for row, ch in enumerate(channels):
    df_filter = df[filters[ch]['ampl_cut']]

    # Extract data
    x_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.ampl']
    y_filtered = df_filter[f'peakparam_{ch}/peakparam_{ch}.echargefit']

    df_saturation = df[filters[ch]['ampl_cut'] & ~filters[ch]['saturation_cut']]
    x_saturation = df_saturation[f'peakparam_{ch}/peakparam_{ch}.ampl']
    y_saturation = df_saturation[f'peakparam_{ch}/peakparam_{ch}.echargefit']

    # First scatter plot (Amplitude vs e-peak Charge with rejection line)
    up_slope, up_offset = 50, 0.5
    up_rejects_filter = y_filtered > line(x_filtered, up_slope, up_offset)
    up_rejects_x = x_filtered[up_rejects_filter]
    up_rejects_y = y_filtered[up_rejects_filter]

    axs[row][0].scatter(x_filtered, y_filtered, s=0.6, color='black', label=f'{ch} - MM Pad')
    axs[row][0].scatter(up_rejects_x, up_rejects_y, s=2, color='blue', label='Rejects')
    axs[row][0].scatter(x_saturation, y_saturation, s=2, color='red', label=f'Saturation {mm_channel}')
    axs[row][0].plot(np.array(x_filtered), line(np.array(x_filtered), up_slope, up_offset), color='red', label='y=x')

    axs[row][0].set_xlabel('Amplitude [V]')
    axs[row][0].set_ylabel('e-peak Charge [pC]')
    axs[row][0].set_xlim(0, 0.45)
    axs[row][0].set_ylim(0, 40)
    axs[row][0].legend()
    axs[row][0].set_title(f'{ch} - Amplitude vs e-peak Charge')

    # Second scatter plot (Amplitude vs Total Charge)
    axs[row][1].scatter(x_filtered, y_filtered, s=0.6, color='black', label=f'{ch} - MM Pad')
    axs[row][1].scatter(x_saturation, y_saturation, s=2, alpha=0.2, color='red', label=f'Saturation {mm_channel}')

    axs[row][1].set_xlabel('Amplitude [V]')
    axs[row][1].set_ylabel('Total Charge [pC]')
    axs[row][1].set_xlim(0, 0.9)
    axs[row][1].set_ylim(0, 500)
    axs[row][1].legend()
    axs[row][1].set_title(f'{ch} - Amplitude vs Total Charge')

    # Third scatter plot (e-peak Charge vs Total Charge)
    x_filtered_2 = df_filter[f'peakparam_{ch}/peakparam_{ch}.echargefit']
    y_filtered_2 = df_filter[f'peakparam_{ch}/peakparam_{ch}.totcharge']

    x_saturation_2 = df_saturation[f'peakparam_{ch}/peakparam_{ch}.echargefit']
    y_saturation_2 = df_saturation[f'peakparam_{ch}/peakparam_{ch}.totcharge']


    axs[row][2].scatter(x_filtered_2, y_filtered_2, s=0.6, color='black', label=f'{ch} - MM Pad')
    axs[row][2].scatter(x_saturation_2, y_saturation_2, s=2, alpha=0.2, color='red', label=f'Saturation {mm_channel}')
    axs[row][2].set_xlabel('e-peak Charge [pC]')
    axs[row][2].set_ylabel('Total Charge [pC]')
    axs[row][2].set_xlim(0, 80)
    axs[row][2].set_ylim(0, 500)
    axs[row][2].legend()
    axs[row][2].set_title(f'{ch} - e-peak Charge vs Total Charge')

plt.tight_layout()
plt.show()

# Get the pad center using the charge weighted method

In [ ]:
channel = mcp_channel
df_filter = df[filters[channel]['charge_cut'] & filters[channel]['xy_cut']]
xs_mcp_filtered = df_filter[f'hitX_{channel}']
ys_mcp_filtered = df_filter[f'hitY_{channel}']
charges_mcp_filtered = df_filter[f'peakparam_{channel}/peakparam_{channel}.charge']
get_pad_center(charges_mcp_filtered, xs_mcp_filtered, ys_mcp_filtered, bin_width=mcp_2d_bin_width, min_tracks_per_2d_bin=0, min_avg_charge_per_2d_bin=0, plot=True, plot_only=True)

In [ ]:
mcp_pad_center_measures = get_pad_center(charges_mcp_filtered, xs_mcp_filtered, ys_mcp_filtered, bin_width=mcp_2d_bin_width, min_tracks_per_2d_bin=mcp_min_tracks_per_2d_bin, min_avg_charge_per_2d_bin=mcp_min_avg_charge_per_2d_bin, plot=True, plot_only=False)
print(f'C1 Pad Center: ({", ".join([str(xi) for xi in mcp_pad_center_measures])})')

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut']]
xs_mm_filtered = df_filter[f'hitX_{mm_channel}']
ys_mm_filtered = df_filter[f'hitY_{mm_channel}']
charges_mm_filtered = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
get_pad_center(charges_mm_filtered, xs_mm_filtered, ys_mm_filtered, bin_width=mm_2d_bin_width, min_tracks_per_2d_bin=0, min_avg_charge_per_2d_bin=0, plot=True, plot_only=True)


In [ ]:
mm_pad_center_measures = get_pad_center(charges_mm_filtered, xs_mm_filtered, ys_mm_filtered, bin_width=mm_2d_bin_width, min_avg_charge_per_2d_bin=mm_min_avg_charge_per_2d_bin, min_tracks_per_2d_bin=mm_min_tracks_per_2d_bin, plot=True)
print(f'{mm_channel} Pad Center: ({", ".join([str(xi) for xi in mm_pad_center_measures])})')

df[f'hitR_{mm_channel}'] = np.sqrt((df[f'hitX_{mm_channel}'] - mm_pad_center_measures[0].val) ** 2 + (df[f'hitY_{mm_channel}'] - mm_pad_center_measures[1].val) ** 2)
r_inner = 4.3
r_outer = hex_radius_inner_to_outer(r_inner)
filters[mm_channel].update({'r_cut': df[f'hitR_{mm_channel}'] < r_outer})

# Get the time walk parameterization raw data

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']

indiv_params, med_bins_params, gaus_bins_params = get_time_walk(time_diff, charges, time_walk_func, p0_time_walk, percentile_cut=(2, 98), plot=True, plot_indiv_fits=False)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
binning_t20_diff = np.linspace(np.min(time_diff), np.max(time_diff), 100)
ax.hist(time_diff, bins=binning_t20_diff, histtype='step', color='black')
#make y-axis log scale
ax.set_yscale('log')
ax.set_xlabel('SAT Raw [ns]')

# Create an inset axis
ax_inset = inset_axes(ax, width="50%", height="70%", loc="upper right")  # % of parent axis

# Inset plot data
binning_t20_diff_nom = np.arange(0, 12, 0.2)
ax_inset.set_yscale('log')
ax_inset.hist(time_diff, bins=binning_t20_diff_nom, histtype='step', color='black')

In [ ]:
# plot SAT vs charge for different r on the same plot
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_na_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']
rs = df_filter[f'hitR_{mm_channel}']

get_time_walk_r_separated(time_diff, charges, rs, percentile_cut=(2,98), binning_type='equal_stats', n_bins=20)
get_time_walk_r_separated(time_diff, charges, rs, percentile_cut=(2,98), binning_type='equal_stats', n_bins=20)

# Time walk parameterization with radial cut

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['r_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_na_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']
xs = df_filter[f'hitX_{mm_channel}']
ys = df_filter[f'hitY_{mm_channel}']
rs = df_filter[f'hitR_{mm_channel}']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
binning_t20_diff = np.linspace(np.min(time_diff), np.max(time_diff), 100)
ax.hist(time_diff, bins=binning_t20_diff, histtype='step', color='black')
#make y-axis log scale
ax.set_yscale('log')
ax.set_xlabel('SAT Raw [ns]')

# Create an inset axis
ax_inset = inset_axes(ax, width="50%", height="70%", loc="upper left")  # % of parent axis

# Inset plot data
binning_t20_diff_nom = np.arange(-3, 12, 0.2)
# ax_inset.axvline(2.5, color='red', ls='-')
# ax_inset.axvline(4.3, color='red', ls='-')
ax_inset.set_yscale('log')
ax_inset.hist(time_diff, bins=binning_t20_diff_nom, histtype='step', color='black')

# Check outliers in the time difference with their track position

In [ ]:
# time_diff_mask = (time_diff < 2.5) | ( time_diff > 4.3)
time_diff_mask = (time_diff < 5) | ( time_diff > 7)
# print(time_diff_mask)
# small_time_diff_mask = time_diff_filter < 2.5
# local_time_diff_mask = np.abs(time_diff_filter) > 4.3 or 7.5

xs_large_time_diff = xs[time_diff_mask]
ys_large_time_diff = ys[time_diff_mask]

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(xs_large_time_diff, ys_large_time_diff, alpha=0.5, label='Outliers')
ax.scatter(mm_pad_center_measures[0].val, mm_pad_center_measures[1].val,
           color='red', marker='x', s=100, label='Pad Center')

center_x, center_y = mm_pad_center_measures[0].val, mm_pad_center_measures[1].val
inner_circle = patches.Circle((center_x, center_y), r_inner, fill=False, edgecolor='blue', linestyle='dashed', label='Inner Circle')
outer_circle = patches.Circle((center_x, center_y), r_outer, fill=False, edgecolor='green', linestyle='solid', label='Outer Circle')

ax.add_patch(inner_circle)
ax.add_patch(outer_circle)

ax.set_xlim(center_x - r_outer - 1, center_x + r_outer + 1)
ax.set_ylim(center_y - r_outer - 1, center_y + r_outer + 1)

ax.set_xlabel('X [mm]')
ax.set_ylabel('Y [mm]')
ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
plt.tight_layout()

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['r_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_na_cut'] & filters[mm_channel]['time_diff_range_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']
xs = df_filter[f'hitX_{mm_channel}']
ys = df_filter[f'hitY_{mm_channel}']
rs = df_filter[f'hitR_{mm_channel}']
# plot SAT vs charge for different r on the same plot
time_walk_nbins = 80
get_time_walk_r_separated(time_diff, charges, rs, percentile_cut=(2,98), binning_type='equal_stats', n_bins=time_walk_nbins, ylim=(-1.25,-0.75))

In [ ]:
time_walk_nbins = 60
indiv_params, med_bins_params, gaus_bins_params = get_time_walk(time_diff, charges, time_walk_func, p0_time_walk, percentile_cut=(2, 98), binning_type='equal_stats', n_bins=time_walk_nbins, plot=True, plot_indiv_fits=False)


In [ ]:
get_time_walk(time_diff, charges, time_walk_func, p0_time_walk, percentile_cut=(2, 98), binning_type='equal_steps', plot=True, plot_indiv_fits=False)

# Time walk correction using Indiv point fit, Median of Bin Fit and Gaus of Bin Fit

In [ ]:
cor_names = ['Individual Points Fit', 'Median of Bin Fit', 'Gaus Fit of Bin Fit']
for cor_meases, cor_name in zip([indiv_params, med_bins_params, gaus_bins_params], cor_names):
    if cor_meases is None:
        print(f'{cor_name}: {cor_meases} is None, skipping')
        continue
    print(f'{cor_name}: {cor_meases}')
    cor_popt = [par.val for par in cor_meases]
    time_diff_cor = time_diff - time_walk_func(charges, *cor_popt)

    fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
    axs[0].scatter(charges, time_diff_cor, alpha=0.2, label=cor_name)
    axs[0].axhline(0, color='red', alpha=0.5)
    axs[0].set_xlabel('Charge [ns]')
    axs[0].set_ylabel('SAT [ns]')
    axs[1].hist(time_diff_cor, bins=np.linspace(np.percentile(time_diff_cor, 1), np.percentile(time_diff_cor, 99), 100))
    axs[1].set_xlabel('SAT [ns]')
    axs[0].legend()
    fig.subplots_adjust(wspace=0.3)
    # Set ylim to 2nd and 98 percentile
    axs[0].set_ylim(np.percentile(time_diff_cor, 1), np.percentile(time_diff_cor, 99))

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(rs, time_diff_cor, alpha=0.2, label=cor_name)
    ax.set_xlabel('Radial Distance [mm]')
    ax.set_ylabel('SAT [ns]')
    ax.axvline(r_inner, color='green', ls='--', label='Hex Inner Radius')
    ax.axvline(r_outer, color='orange', ls='--', label='Hex Outer Radius')
    ax.legend()
    ax.set_ylim(-2, 2)
    ax.set_xlim(-1, 10)

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['r_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_na_cut'] & filters[mm_channel]['time_diff_range_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']

charge_bins = np.percentile(charges, np.linspace(0, 100, n_charge_bins + 1))
charge_bins = [[charge_bins[i], charge_bins[i+1]]for i in range(len(charge_bins) - 1)]

time_diff_cors = {
    'Uncorrected': time_diff,
    'Corrected Gaus': time_diff - time_walk_func(charges, *[par.val for par in gaus_bins_params]),
}
if med_bins_params:
    time_diff_cors.update({'Corrected Median': time_diff - time_walk_func(charges, *[par.val for par in med_bins_params])})
time_diff_cor_colors = {'Uncorrected': 'k', 'Corrected Median': 'm', 'Corrected Gaus': 'c'}
charge_scan_resolutions = {}

for cor_name, time_diff_cor_i in time_diff_cors.items():
    charge_reses, charge_means, charge_events = get_charge_scan(time_diff_cor_i, charges, charge_bins, ns_to_ps=True, percentile_cuts=(5, 95), plot=False)
    charge_scan_resolutions[cor_name] = charge_reses

In [ ]:
resol_vs_charge_popt = None

fig, ax = plt.subplots(figsize=(8, 5))

charge_bin_means, charge_bin_stds = [], []
for charge_bin in charge_bins:
    charge_bin_low, charge_bin_high = charge_bin
    mask = (charge_bin_low <= charges) & (charges < charge_bin_high)
    charge_bin_means.append(np.mean(charges[mask]))
    charge_bin_stds.append(np.std(charges[mask]))

for cor_name in time_diff_cors.keys():
    charge_res_vals = [x.val for x in charge_scan_resolutions[cor_name]]
    charge_res_errs = [x.err for x in charge_scan_resolutions[cor_name]]
    ax.errorbar(charge_bin_means, charge_res_vals, xerr=charge_bin_stds, yerr=charge_res_errs, ls='none', marker='.', color=time_diff_cor_colors[cor_name], label=cor_name, alpha=0.5)
    if cor_name == 'Corrected Gaus':
        xdata = np.array(charge_bin_means)
        ydata = np.array([x.val for x in charge_scan_resolutions['Corrected Gaus']])
        yerr = np.array([x.err for x in charge_scan_resolutions['Corrected Gaus']])

        nan_filter = np.isfinite(ydata)
        xdata, ydata, yerr = xdata[nan_filter], ydata[nan_filter], yerr[nan_filter]

        x_fit = np.linspace(min(xdata), max(xdata), 500)
        ax.plot(x_fit, resol_func(x_fit, *p0_resol), color='gray', label='Guess')

        popt, pcov = cf(resol_func, xdata, ydata, sigma=yerr, absolute_sigma=True, p0=p0_resol, maxfev=10000)

        if cor_name == resolution_correction_to_use:
            resol_vs_charge_popt = popt

        y_fit = resol_func(x_fit, *popt)

        ax.plot(x_fit, y_fit, 'r--', label='Double Exp Fit (Corrected Gaus)')
ax.set_ylim(bottom=0)
ax.legend()
ax.set_xlabel('Total Charge (pC)')
ax.set_ylabel('Time Resolution (ps)')

# Correct for timewalk for the best parameterization


# Get the time difference distribution in rings

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_range_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']
xs = df_filter[f'hitX_{mm_channel}']
ys = df_filter[f'hitY_{mm_channel}']
rs = df_filter[f'hitR_{mm_channel}']

# time_diff_cor = time_diff - time_walk_func(charges, *[x.val for x in indiv_params])
# time_diff_cor = time_diff - time_walk_func(charges, *[x.val for x in med_bins_params])
time_diff_cor = time_diff - time_walk_func(charges, *[x.val for x in gaus_bins_params])
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(rs, time_diff_cor, alpha=0.2)
ax.set_xlabel('Radial Distance [mm]')
ax.set_ylabel('SAT [ns]')
ax.axhline(0, color='gray', zorder=0)
ax.set_ylim(-1.5, 1)

In [ ]:
def get_ring_scan(time_diff_cor, rings, ring_bin_width, rs, percentile_cuts=(None, None), nsigma_filter=None, xs=None, ys=None, pad_center=None, shape='circle', plot=False):

    if shape == 'square' and (xs is None or ys is None or pad_center is None) or shape not in ['circle', 'square']:
        print(f"Shape is square, but xs and ys are not provided or unknown shape {shape}. Cannot plot.")
        return
    if plot:
        if xs is not None and ys is not None:
            fig, axs = plt.subplots(ncols=2, figsize=(12, 6))
            ax, ax_xy = axs
            ax_xy.set_xlabel('X [mm]')
            ax_xy.set_ylabel('Y [mm]')
        else:
            fig, ax = plt.subplots(figsize=(8, 5))
        ax.scatter(rs, time_diff_cor, alpha=0.2)
        ax.set_xlabel('Radial Distance from Pad Center [mm]')
        ax.set_ylabel('SAT [ns]')
        for r_bin_edge in rings:
            ax.axvline(r_bin_edge, color='red')
            ax.set_xlim(0, 10)
            ax.set_ylim(-2, 2)

            if xs is not None and ys is not None:
                pass
                # Make circle patch at r_bin_edge
                # ax_xy.add_patch(plt.Circle((center_x, center_y), r_bin_edge, color='red', fill=False, alpha=0.5))

    time_diff_binning = 100
    r_bin_centers, time_resolutions, mean_sats = [], [], []
    for r_bin_edge in rings:
        r_bin_upper_edge = r_bin_edge + ring_bin_width
        if shape == 'circle':
            r_bin_filter = (rs > r_bin_edge) & (rs <= r_bin_upper_edge)
        elif shape == 'square':  # Check if xs and ys are in the square region
            pad_center_x, pad_center_y = pad_center[0].val, pad_center[1].val
            square_distances = np.maximum(np.abs(xs - pad_center_x), np.abs(ys - pad_center_y))
            r_bin_filter = (square_distances > r_bin_edge) & (square_distances <= r_bin_upper_edge)
        time_diffs_r_bin = time_diff_cor[r_bin_filter]
        print(len(time_diffs_r_bin))

        time_diffs_r_bin = make_percentile_cuts(time_diffs_r_bin, percentile_cuts)

        fit_meases, time_hist, time_diff_bin_centers, time_hist_err = fit_time_diffs(time_diffs_r_bin, n_bins=time_diff_binning, nsigma_filter=nsigma_filter, min_events=100, return_hist=True)
        mean_sats.append(fit_meases[1]*1e3)
        time_resolutions.append(fit_meases[2] * 1e3)
        r_bin_centers.append((r_bin_edge + r_bin_upper_edge) / 2)

        if plot:
            fig_ring, ax_ring = plt.subplots(figsize=(8, 5))

            # Only plot bins with non-zero entries
            mask_nonzero = time_hist > 0
            ax_ring.errorbar(time_diff_bin_centers[mask_nonzero], time_hist[mask_nonzero],
                             yerr=time_hist_err[mask_nonzero], fmt='o', color='black',
                             label=f'{r_bin_edge:.2f} - {r_bin_upper_edge:.2f} mm')

            fit_str = rf'$A = {fit_meases[0]}$' + '\n' + rf'$\mu = {fit_meases[1]}$' + '\n' + rf'$\sigma = {fit_meases[2]}$'
            ax_ring.plot(time_diff_bin_centers, gaus(time_diff_bin_centers, *[par.val for par in fit_meases]), color='red', label='Fit')
            ax_ring.set_xlabel('SAT [ns]')
            ax_ring.annotate(
            fit_str,
            xy=(0.1, 0.9), xycoords='axes fraction',
            ha='left', va='top',
            bbox=dict(boxstyle='round,pad=0.5', edgecolor='black', facecolor='lightyellow')
            )
            ax_ring.legend()

            if xs is not None and ys is not None:
                xs_ring, ys_ring = xs[r_bin_filter], ys[r_bin_filter]
                ax_xy.scatter(xs_ring, ys_ring, alpha=0.2)

    if plot:
        fig, ax = plt.subplots(figsize=(8, 5))
        r_bin_width = rings[1] - rings[0]
        ax.errorbar(r_bin_centers, [x.val for x in time_resolutions], yerr=[x.err for x in time_resolutions],
                    xerr=r_bin_width / 2, marker='o', color='black', ls='none')
        ax.set_xlabel('Radial Distance from Pad Center [mm]')
        ax.set_ylabel('Time Resolution [ps]')
        ax.set_ylim(bottom=0)

        fig, ax = plt.subplots(figsize=(8, 5))
        r_bin_width = rings[1] - rings[0]
        ax.errorbar(r_bin_centers, [x.val for x in mean_sats], yerr=[x.err for x in mean_sats],
                    xerr=r_bin_width / 2, marker='o', color='black', ls='none')
        ax.set_xlabel('Radial Distance from Pad Center [mm]')
        ax.set_ylabel('SAT [ps]')

    return r_bin_centers, time_resolutions, mean_sats

In [ ]:
r_bin_width = 2.5
r_bin_edges = np.arange(0, 8, r_bin_width)
center_x, center_y = mm_pad_center_measures[0].val, mm_pad_center_measures[1].val
get_ring_scan(time_diff_cor, r_bin_edges, r_bin_width, rs, percentile_cuts=(1,99), nsigma_filter=3, xs=xs, ys=ys, plot=True)
get_ring_scan(time_diff_cor, r_bin_edges, r_bin_width, rs, percentile_cuts=(1,99), nsigma_filter=3, xs=xs, ys=ys, pad_center=mm_pad_center_measures, shape='square', plot=True)

# Monitor the radial distribution of the charge

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(rs, charges, alpha=0.2)
ax.axhline(0, color='gray', zorder=0)
ax.axvline(r_inner, color='green', zorder=10)
ax.axvline(r_outer, color='orange', zorder=10)
ax.set_xlim(-1, 10)

# Get the time difference distribution in circles overlapping to get resolution and SAT map

In [ ]:
x_pad_center, y_pad_center = mm_pad_center_measures
scan_radius = 1.0  # mm Radius of scanning circle
min_events_per_circle = 350  # Minimum number of events per scan circle. If less, skip the step
xy_scan = mm_pad_center_measures
# Get center of pad
x_center = x_pad_center.val
y_center = y_pad_center.val
n_steps = 10
step_size = 2 * scan_radius / n_steps # 0.2mm
xs_scan = np.linspace(x_center - scan_radius, x_center + scan_radius, n_steps + 1)
ys_scan = np.linspace(y_center - scan_radius, y_center + scan_radius, n_steps + 1)
# y_scan = y_center - scan_radius
# xy_pairs = [(x, y_scan) for x in xs_scan]
xy_pairs = [(x, y) for x in xs_scan for y in ys_scan]  
scan_resolutions, scan_means, events = get_circle_scan(time_diff_cor, xs, ys, xy_pairs, radius=scan_radius, min_events=min_events_per_circle, ns_to_ps=True, percentile_cuts=(2, 98), plot=False)

In [ ]:
y_values = sorted(set(y for x, y in xy_pairs))  # Extract unique y values

# Prepare the plot
fig, ax = plt.subplots(figsize=(10, 6))

for y in y_values:
    # Extract the scan_resolutions corresponding to the current y value
    xs_for_y = [x-x_center for x, y_ in xy_pairs if y_ == y]  # x values corresponding to this y
    scan_resolutions_for_y = [scan_resolutions[i].val for i, (x, y_) in enumerate(xy_pairs) if y_ == y]
    scan_resolutions_for_y_err = [scan_resolutions[i].err for i, (x, y_) in enumerate(xy_pairs) if y_ == y]

    # Plot scan_resolutions against xs for this y value
    ax.errorbar(xs_for_y, scan_resolutions_for_y, yerr=scan_resolutions_for_y_err, ls='none', marker='o', label=f'y = {y:.2f} mm', alpha=0.5)

ax.set_xlabel('Distance from pad center in x [mm]')
ax.set_ylabel('Resolution (ps) ')
# ax.set_title('Scan Resolutions vs X values for different Y values')
ax.set_ylim(bottom=15)
ax.legend()
ax.grid(True)

x_values = sorted(set(x for x, y in xy_pairs))  # Extract unique y values

# Prepare the plot
fig, ax = plt.subplots(figsize=(10, 6))

for x in x_values:
    # Extract the scan_resolutions corresponding to the current y value
    ys_for_x = [y-y_center for x_, y in xy_pairs if x_ == x]  # x values corresponding to this y
    scan_resolutions_for_x = [scan_resolutions[i].val for i, (x_, y) in enumerate(xy_pairs) if x_ == x]
    scan_resolutions_for_x_err = [scan_resolutions[i].err for i, (x_, y) in enumerate(xy_pairs) if x_ == x]

    # Plot scan_resolutions against xs for this y value
    ax.errorbar(ys_for_x, scan_resolutions_for_x, yerr=scan_resolutions_for_x_err, ls='none', marker='o', label=f'x = {x:.2f} mm', alpha=0.5)

ax.set_xlabel('Distance from pad center in y [mm]')
ax.set_ylabel('Resolution (ps)')
# ax.set_title('Scan Resolutions vs Y values for different X values')
ax.set_ylim(bottom=15)
ax.legend()
ax.grid(True)

In [ ]:
# y_values = sorted(set(y for x, y in xy_pairs))  # Extract unique y values

# Prepare the plot
fig, ax = plt.subplots(figsize=(10, 6))

for y in y_values:
    # Extract the scan_resolutions corresponding to the current y value
    xs_for_y = [x-x_center for x, y_ in xy_pairs if y_ == y]  # x values corresponding to this y
    scan_means_for_y = [scan_means[i].val for i, (x, y_) in enumerate(xy_pairs) if y_ == y]
    scan_means_for_y_err = [scan_means[i].err for i, (x, y_) in enumerate(xy_pairs) if y_ == y]

    # Plot scan_resolutions against xs for this y value
    ax.errorbar(xs_for_y, scan_means_for_y, yerr=scan_means_for_y_err, ls='none', marker='o', label=f'y = {y:.2f} mm', alpha=0.5)

ax.set_xlabel('Distance from pad center in x [mm]')
ax.set_ylabel('SAT (ps) ')
# ax.set_title('Scan Resolutions vs X values for different Y values')
# ax.set_ylim(bottom=0)
ax.legend()
ax.grid(True)

# x_values = sorted(set(x for x, y in xy_pairs))  # Extract unique y values

# Prepare the plot
fig, ax = plt.subplots(figsize=(10, 6))

for x in x_values:
    # Extract the scan_resolutions corresponding to the current y value
    ys_for_x = [y-y_center for x_, y in xy_pairs if x_ == x]  # x values corresponding to this y
    scan_means_for_y = [scan_means[i].val for i, (x_, y) in enumerate(xy_pairs) if x_ == x]
    scan_means_for_y_err = [scan_means[i].err for i, (x_, y) in enumerate(xy_pairs) if x_ == x]

    # Plot scan_resolutions against xs for this y value
    ax.errorbar(ys_for_x, scan_means_for_y, yerr=scan_means_for_y_err, ls='none', marker='o', label=f'x = {x:.2f} mm', alpha=0.5)

ax.set_xlabel('Distance from pad center in y [mm]')
ax.set_ylabel('SAT (ps)')
# ax.set_title('Scan Resolutions vs Y values for different X values')
# ax.set_ylim(bottom=0)
ax.legend()
ax.grid(True)

# Get the 2D heatmaps for timing resolution and SAT

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12, 15))
axs = axs.flatten()
angles = [0, 30, 45, 60, 90, 120, 135, 150]  # degrees
for idx, angle in enumerate(angles):
    xy_pairs = generate_line_scan(x_center, y_center, scan_radius, n_steps, angle)

    scan_resolutions, scan_means, events = get_circle_scan(
        time_diff_cor, xs, ys, xy_pairs,
        radius=scan_radius,
        min_events=min_events_per_circle,
        ns_to_ps=True,
        percentile_cuts=(2, 98),
        shape='square',
        plot=False
    )

    # Relative position along the line, in mm
    ts = np.linspace(-scan_radius, scan_radius, len(scan_means))  # line distances
    means = [m.val for m in scan_means]
    errors = [m.err for m in scan_means]

    ax = axs[idx]
    ax.errorbar(ts, means, yerr=errors, fmt='o', label=f"{angle}°", alpha=0.8)
    # ax.set_title(f"Mean vs. Position at {angle}°")
    # ax.set_xlabel("Relative position along scan line (mm)")
    ax.set_ylabel("SAT (ps)")
    ax.grid(True)
    ax.legend()

 # Remove all individual x-axis labels
    ax.set_xlabel("Relative Position along scan line")

plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12, 15))
axs = axs.flatten()
angles = [0, 30, 45, 60, 90, 120, 135, 150]  # degrees
for idx, angle in enumerate(angles):
    xy_pairs = generate_line_scan(x_center, y_center, scan_radius, n_steps, angle)

    scan_resolutions, scan_means, events = get_circle_scan(
        charges, xs, ys, xy_pairs,
        radius=scan_radius,
        min_events=min_events_per_circle,
        ns_to_ps=True,
        percentile_cuts=(2, 98),
        shape='square',
        plot=False
    )

    # Relative position along the line, in mm
    ts = np.linspace(-scan_radius, scan_radius, len(scan_means))  # line distances
    means = [m.val for m in scan_means]
    errors = [m.err for m in scan_means]

    ax = axs[idx]
    ax.errorbar(ts, means, yerr=errors, fmt='o', label=f"{angle}°", alpha=0.8)
    # ax.set_title(f"Mean vs. Position at {angle}°")
    # ax.set_xlabel("Relative position along scan line (mm)")
    ax.set_ylabel("Total charge (ps)")
    ax.grid(True)
    ax.legend()

    # Remove all individual x-axis labels
    ax.set_xlabel("Relative Position along scan line")

plt.tight_layout()
plt.show()

In [ ]:
n_steps_2D = 100

scan_xs = np.linspace(x_center - r_outer * 2.0, x_center + r_outer * 2.0, n_steps_2D + 1)
scan_ys = np.linspace(y_center - r_outer * 2.0,  y_center + r_outer * 2.0, n_steps_2D + 1)

# Generate mesh grid
x_mesh, y_mesh = np.meshgrid(scan_xs, scan_ys)
xy_pairs = list(zip(x_mesh.ravel(), y_mesh.ravel()))  # Flatten to list of (x, y) pairs

scan_radius=2.0
scan_resolutions, scan_means, events = get_circle_scan(time_diff_cor, xs, ys, xy_pairs, radius=scan_radius, min_events=min_events_per_circle, ns_to_ps=True, percentile_cuts=(2, 98), shape='square', plot=False)

plot_2D_circle_scan(scan_resolutions, scan_means, scan_xs, scan_ys, scan_events=events, radius=scan_radius, percentile_filter=(1, 99))

In [ ]:
df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut']]
charges = df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge']
time_diff = df_filter[f'{mm_channel}_time_diff']
xs = df_filter[f'hitX_{mm_channel}']
ys = df_filter[f'hitY_{mm_channel}']

n_steps_2D = 100
# Define scan ranges
# scan_xs = np.arange(20, 50, 0.5)
# scan_ys = np.arange(20, 50, 0.5)  # Define y range

scan_xs = np.linspace(x_center - r_outer * 2.0, x_center + r_outer * 2.0, n_steps_2D + 1)
scan_ys = np.linspace(y_center - r_outer * 2.0,  y_center + r_outer * 2.0, n_steps_2D + 1)

# Generate mesh grid
x_mesh, y_mesh = np.meshgrid(scan_xs, scan_ys)
xy_pairs = list(zip(x_mesh.ravel(), y_mesh.ravel()))  # Flatten to list of (x, y) pairs

# Convert to 2D arrays
scan_resolution_vals = [res.val for res in scan_resolutions]
scan_resolutions_2d = np.array(scan_resolution_vals).reshape(len(scan_ys), len(scan_xs))

levels = np.linspace(0, 100, 50)

fig, ax = plt.subplots(figsize=(8, 6))
# contour = ax.contourf(x_mesh, y_mesh, scan_resolutions_2d, levels=50, cmap="jet", vmin=res_vmin, vmax=res_vmax)
contour = ax.contourf(x_mesh, y_mesh, scan_resolutions_2d, levels=levels, cmap="jet")

# Add color bar
cbar = plt.colorbar(contour)
cbar.set_label("Timing Resolution [ps]")

# Labels and title
ax.set_xlabel("X Position [mm]")
ax.set_ylabel("Y Position [mm]")
ax.set_title(f"Timing Resolution Contour Plot")

# 2D scan for the non corrected SAT

In [ ]:
scan_xs = np.linspace(x_center - r_outer * 2.0, x_center + r_outer * 2.0, n_steps_2D + 1)
scan_ys = np.linspace(y_center - r_outer * 2.0,  y_center + r_outer * 2.0, n_steps_2D + 1)

# Generate mesh grid
x_mesh, y_mesh = np.meshgrid(scan_xs, scan_ys)
xy_pairs = list(zip(x_mesh.ravel(), y_mesh.ravel()))  # Flatten to list of (x, y) pairs

scan_resolutions, scan_means, events = get_circle_scan(time_diff, xs, ys, xy_pairs, radius=scan_radius, min_events=min_events_per_circle, ns_to_ps=True, percentile_cuts=(2, 98), shape='square', plot=False)

plot_2D_circle_scan(scan_resolutions, scan_means, scan_xs, scan_ys, scan_events=events, radius=scan_radius)

In [ ]:
# Write relevant event data to dataframe
out_df_path = f'scan_pad_dfs/Run{run_number}-Pool{pool_number}-{mm_channel}.csv'

df_filter = df[filters[mm_channel]['totcharge_cut'] & filters[mm_channel]['xy_cut'] & filters[mm_channel]['saturation_cut'] & filters[mm_channel]['time_diff_na_cut']].copy()

# Get corrected time difference and add to dataframe
time_diff = df_filter[f'{mm_channel}_time_diff']
cor_popt = [par.val for par in gaus_bins_params]

# Normalize to median of charges distribution
# time_diff_cor_norm = time_walk_func(np.median(charges), *cor_popt)
time_diff_cor_norm = 0
time_diff_cor = time_diff - time_walk_func(charges, *cor_popt) + time_diff_cor_norm
df_filter[f'{mm_channel}_time_diff_cor'] = time_diff_cor

# Get estimated time difference resolution and add to dataframe
df_filter[f'{mm_channel}_charge_time_res'] = resol_func(df_filter[f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge'], *resol_vs_charge_popt)

# Put time_diff_filter in df
df_filter['time_diff_range_cut'] = filters[mm_channel]['time_diff_range_cut']

save_cols = {'SRSNo': 'SRSNo',
             'eventNo': 'eventNo',
             f'hitX_{mm_channel}': 'hitX',
             f'hitY_{mm_channel}': 'hitY',
             'time_diff_range_cut': 'time_diff_range_cut',
             f'peakparam_{mm_channel}/peakparam_{mm_channel}.totcharge': 'totcharge',
             f'{mm_channel}_time_diff': 'time_diff',
             f'{mm_channel}_time_diff_cor': 'time_diff_cor',
             f'{mm_channel}_charge_time_res': 'charge_time_res'}
df_filter = df_filter[list(save_cols.keys())]
df_filter = df_filter.rename(columns=save_cols)
df_filter.to_csv(out_df_path, index=False)

# Add pad centers to pad_centers dataframe
scan_pad_centers_df = f'scan_pad_dfs/scan_pad_centers.csv'
x_c, y_c = mm_pad_center_measures
update_pad_centers_csv(scan_pad_centers_df, run_number, pool_number, mm_channel, x_c.val, x_c.err, y_c.val, y_c.err)
